# Long Read Analysis:

#### Assembly Stage:
##### Overall journey and basic instructions in how to get started with using  Canu, Falcon and Metaspades.

## Canu, how it works-

### Pipeline:

The canu pipeline, that is, what it actually computes, comprises of computing overlaps and processing the overlaps to some result. Each of the three tasks (read correction, read trimming and unitig construction) follow the same pattern:

1. Load reads into the read database, gkpStore.
2. Compute k-mer counts in preparation for the overlap computation.
3. Compute overlaps.
4. Load overlaps into the overlap database, ovlStore.
5. Do something interesting with the reads and overlaps.

    -The read correction task will replace the original noisy read sequences with consensus sequences computed from overlapping reads.
    
    -The read trimming task will use overlapping reads to decide what regions of each read are high-quality sequence, and what regions should be trimmed. 
    
    -After trimming, the single largest high-quality chunk of sequence is retained.
    
    -The unitig construction task finds sets of overlaps that are consistent, and uses those to place reads into a multialignment layout. 
    
    -The layout is then used to generate a consensus sequence for the unitig.

## Canu, usage-

## Activate conda environment:

In [ ]:
conda activate long-read-assembly # activate 

If the above doesn't work, try the following:

In [ ]:
conda activate long-read-assembly # activate 

[tali01@bradysaurus ~]$ conda activate /home/tali01/anaconda3/envs/long-read-assembly/ 
(long-read-assembly) [tali01@bradysaurus ~]$

## Test tools in conda environment:

In [ ]:
(long-read-assembly) [tali01@bradysaurus ~]$  canu ## call canu

usage: canu [-correct | -trim | -assemble | -trim-assemble] \
            [-s <assembly-specifications-file>] \
             -p <assembly-prefix> \
             -d <assembly-directory> \
             genomeSize=<number>[g|m|k] \
             errorRate=0.X \
            [other-options] \
            [-pacbio-raw | -pacbio-corrected | -nanopore-raw | -nanopore-corrected] *fastq

## Run tools in conda environment

In [ ]:
nohup canu  -p ecoli -d /nasdata/Neev/simlord/sim/canu_test_output/ecoli-pacbio  genomeSize=4.8m gnuplotTested=true  -pacbio-raw /nasdata/Neev/simlord/tool_sample_data/canu/pacbio.fastq > canu_conda_sample_nohup.out &

-The -p option, to set the file name prefix of intermediate and output files, is mandatory. 

-If -d is not supplied, canu will run in the current directory, otherwise, Canu will create the assembly-directory and run in that directory. It is _not_ possible to run two different assemblies in the same directory.

-One parameter is required: the genomeSize (in bases, with common SI prefixes allowed, for example, 4.7m or 2.8g).
The genome size estimate is used to decide how many reads to correct (via the corOutCoverage parameter) and how sensitive the mhap overlapper should be (via the mhapSensitivity parameter). It also impacts some logging, in particular, reports of NG50 sizes.



In [ ]:
 nohup canu -p single_cosmid -d /nasdata/Neev/simlord/canu_test_output/single_cosmid genomeSize=4.8m gnuplotTested=true -pacbio-raw /nasdata/Neev/simlord/sim/sim_cosmids/single_cosmid_SIM.fastq > canu_conda_single_nohup.out

In [ ]:
nohup canu -p 3_cosmid -d /nasdata/Neev/simlord/canu_test_output/3_cosmids genomeSize=4.8m gnuplotTested=true -pacbio-raw /nasdata/Neev/simlord/sim/sim_cosmids/3_cosmids.fastq.fastq > canu_conda_three_nohup.out

In [ ]:
 nohup canu -p insert_cosmid -d /nasdata/Neev/simlord/canu_test_output/insert_cosmid genomeSize=4.8m gnuplotTested=true -pacbio-raw /nasdata/Neev/simlord/sim/sim_cosmids/all_reads_clipped_vector.fq > canu_conda_insert_nohup.out

### Other options

## Parallelizing tool runs in conda environment

In [ ]:
(long-read-assembly) [tali01@bradysaurus sim_cosmids]$ ls *.fastq
3_cosmids.fastq  all_reads_clipped_vector.fastq  single_cosmid_SIM.fastq

In [ ]:
(long-read-assembly) [tali01@bradysaurus sim_cosmids]$ ls *fastq | awk 'BEGIN {FS=".fastq"} ; {print "nohup canu  -p "$1" -d /nasdata/Neev/simlord/canu_test_output/canu_test_parallelize/"$1"  genomeSize=4.8m gnuplotTested=true  -pacbio-raw /nasdata/Neev/simlord/sim/sim_cosmids/"$0" > /home/tali01/parallelize/canu_conda_"$1"_nohup.out &"}'
nohup canu  -p 3_cosmids -d /nasdata/Neev/simlord/canu_test_output/canu_test_parallelize/3_cosmids  genomeSize=4.8m gnuplotTested=true  -pacbio-raw /nasdata/Neev/simlord/sim/sim_cosmids/3_cosmids.fastq > /home/tali01/parallelize/canu_conda_3_cosmids_nohup.out &
nohup canu  -p all_reads_clipped_vector -d /nasdata/Neev/simlord/canu_test_output/canu_test_parallelize/all_reads_clipped_vector  genomeSize=4.8m gnuplotTested=true  -pacbio-raw /nasdata/Neev/simlord/sim/sim_cosmids/all_reads_clipped_vector.fastq > /home/tali01/parallelize/canu_conda_all_reads_clipped_vector_nohup.out &
nohup canu  -p single_cosmid_SIM -d /nasdata/Neev/simlord/canu_test_output/canu_test_parallelize/single_cosmid_SIM  genomeSize=4.8m gnuplotTested=true  -pacbio-raw /nasdata/Neev/simlord/sim/sim_cosmids/single_cosmid_SIM.fastq > /home/tali01/parallelize/canu_conda_single_cosmid_SIM_nohup.out &

In [ ]:
ls *fastq | awk 'BEGIN {FS=".fastq"} ; {print "nohup canu  -p "$1" -d /nasdata/Neev/simlord/canu_test_output/canu_test_parallelize/"$1"  genomeSize=4.8m gnuplotTested=true  -pacbio-raw /nasdata/Neev/simlord/sim/sim_cosmids/"$0" > /home/tali01/parallelize/canu_conda_"$1"_nohup.out &"}' > sim_parallel
(long-read-assembly) [tali01@bradysaurus sim_cosmids]$ sh sim_parallel

There are two modes that canu runs in: locally, using just one machine, or grid-enabled, using multiple hosts managed by a grid engine. In both cases, Canu will auto-detect available resources and configure job sizes based on the resources and genome size you’re assembling. Thus, most users should be able to run the command without modifying the defaults. Some advanced options are outlined below. 

**useGrid<tag\>=boolean
Run this stage on the grid, usually in parallel.**

**gridOptions<tag\>=string
Supply this string to the grid submit command.**

**<tag\>Memory=integer
Use this many gigabytes of memory, per process.**

**<tag\>Threads
Use this many compute threads per process.**

**<tag\>Concurrency
If not on the grid, run this many jobs at the same time.**


<h2 style='padding: 10px'>Runtimes</h2><table class='table table-striped'> <thead> <tr><th>Tool</th>  <th>Sample Data</th>  <th>Single_cosmid Runtime (3.8M) Vector Included</th> <th>3_cosmids Runtime (13M) Vector Included</th>  <th>3_cosmids Runtime (11M) Vector Excluded</th> <th> Total</th>   </tr> </thead><thead> <tr> <th>Canu</th> <th>ecoli_dataset Runtime:  18:02:25 -18:16:11 (13 min 56 sec)</th> <th>12:49:39- 12:54:11  ( 4 minutes 32 seconds)</th>  <th>13:55:36- 13:57:32 (3 min 8 secs)</th> <th>14:31:21-14:37:19 (6 minutes 40 secs)</th> <th>(14min 30 secs) (27.8M)</th> </tr> </thead> <tbody> <tr> <th scope='row'>Canu (parallelization)</th> <td></td> <td>17:11:50 - 17:17:02 (5min 12 secs)</td> <td>17:11:50 -17:19:09 (7min 19 secs)</td> <td>17:11:54 -  17:18:44 (6min 50secs) </td> <th>(7min 19 secs) (27.8M)</th> <th 

## Canu, outputs-


<h2 style='padding: 10px'>Test Outcomes</h2><table class='table table-striped'> <thead> <tr><th>Outputs</th>  <th>Description</th>  <th>Single_cosmid Runtime (3.8M) Vector Included</th> <th>3_cosmids Runtime (13M) Vector Included</th>  <th>3_cosmids Runtime (11M) Vector Excluded</th>   </tr> </thead><thead> <tr> <th>prefix.report</th> <th>Most of the analysis reported during assembly. This will report the histogram of read lengths, the histogram or k-mers in the raw and corrected reads, the summary of corrected data, summary of overlaps, and the summary of contig lengths.</th> <th></th>  <th></th> <th></th>  </tr> </thead> <tbody> <tr> <th scope='row'>prefix.correctedReads.fasta.gz
</th> <td>The reads after correction.</td> <td></td> <td></td> <td> </td> <tr></tr>  </tr> </thead> <tbody> <tr> <th scope='row'>prefix.trimmedReads.fasta.gz
</th> <td>The corrected reads after overlap based trimming.</td> <td></td> <td></td> <td> </td> <tr></tr>   </tr> </thead> <tbody> <tr> <th scope='row'>prefix.contigs.fasta
</th> <td>Everything which could be assembled and is the primary assembly, including both unique and repetitive elements.</td> <td></td> <td></td> <td></td> <tr></tr>  </tr> </thead> <tbody> <tr> <th scope='row'>prefix.unitigs.fasta
</th> <td>Contigs, split at alternate paths in the graph.</td> <td></td> <td></td> <td> </td> <tr></tr> <th scope='row'>prefix.unassembled.fasta
</th> <td>Reads and low-coverage contigs which could not be incorporated into the primary assembly.</td> <td></td> <td></td> <td> </td> <tr></tr>  <th scope='row'>prefix.contigs.gfa
</th> <td>Unused or ambiguous edges between contig sequences. The GFA format cannot represent partial overlaps between contigs (that is part of contig A overlaps with part of contig B).</td> <td></td> <td></td> <td></td> <tr></tr> <th scope='row'>prefix.unitigs.gfa
</th> <td>Since the GFA format cannot represent partial overlaps, the contigs are split at all such overlap junctions into unitigs. The unitigs capture non-branching subsequences within the contigs and will break at any ambiguity (e.g. a haplotype switch).</td> <td></td> <td></td> <td> </td> <tr></tr> <th scope='row'>prefix..unitigs.bed
</th> <td>The position of each unitig in a contig.</td> <td></td> <td></td> <td> </td> <tr></tr> <th scope='row'>prefix.contigs.layout, prefix.unitigs.layout
</th> <td>(undocumented)</td> <td></td> <td></td> <td> </td> <tr></tr> <th scope='row'>prefix.contigs.layout.readToTig, prefix.unitigs.layout.readToTig

</th> <td>The position of each read in a contig (unitig).</td> <td></td> <td></td> <td> </td> <tr></tr> <th scope='row'>prefix.contigs.layout.tigInfo, prefix.unitigs.layout.tigInfo

</th> <td>A list of the contigs (unitigs), lengths, coverage, number of reads and other metadata. Essentially the same information provided in the FASTA header line.</td> <td></td> <td></td> <td> </td> <tr></tr><th 


